In [1]:
import pyspark

In [2]:
spark = pyspark.sql.SparkSession.builder.appName("demo_app") \
        .config("spark.jars.packages","com.microsoft.azure:azure-storage:8.6.3") \
        .config("spark.jars.packages","org.apache.hadoop:hadoop-azure:3.3.2") \
        .getOrCreate()

In [6]:
df = spark.read.json("credit_note_LoadTS=20230209T114426Z.json")

In [9]:
from pyspark.sql import functions as F

In [14]:
df.show()

+--------------------+--------------------+--------------------+--------------+--------------------+------+
|         CreditNotes|         DateTimeUTC|                  Id|  ProviderName| ReplicationDateTime|Status|
+--------------------+--------------------+--------------------+--------------+--------------------+------+
|[{[], null, {[], ...|/Date(16759430843...|2ec55dd3-3f1f-488...|Azure DataLake|2023-02-09T11:44:...|    OK|
|[{[], null, {[], ...|/Date(16759430854...|90c84bdc-193e-438...|Azure DataLake|2023-02-09T11:44:...|    OK|
|[{[{8439.06, /Dat...|/Date(16759430867...|76d0ea9b-6d8e-484...|Azure DataLake|2023-02-09T11:44:...|    OK|
|[{[{7259.16, /Dat...|/Date(16759430882...|eecaa6c6-7b1e-484...|Azure DataLake|2023-02-09T11:44:...|    OK|
|[{[{361.0, /Date(...|/Date(16759430894...|fdfb4b3f-53f5-435...|Azure DataLake|2023-02-09T11:44:...|    OK|
|[{[], null, {[], ...|/Date(16759430904...|5334ddc6-1bbd-41b...|Azure DataLake|2023-02-09T11:44:...|    OK|
|[{[], null, {[], ...|/Date(

In [36]:
df_exp = df.select([F.explode("CreditNotes").alias("jsons"), "ReplicationDateTime"]).select([F.expr("jsons.*"), "ReplicationDateTime"])

In [38]:
df_exp.printSchema()

root
 |-- Allocations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Amount: double (nullable = true)
 |    |    |-- Date: string (nullable = true)
 |    |    |-- Invoice: struct (nullable = true)
 |    |    |    |-- CreditNotes: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- HasErrors: boolean (nullable = true)
 |    |    |    |-- InvoiceAddresses: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- InvoiceID: string (nullable = true)
 |    |    |    |-- InvoiceNumber: string (nullable = true)
 |    |    |    |-- InvoicePaymentServices: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- IsDiscounted: boolean (nullable = true)
 |    |    |    |-- LineItems: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- Overpayments: a

In [37]:
df_exp.show()

+-----------+---------------+--------------------+--------------------+----------------+------------+------------+--------------------+-------------------+-------+-------------+---------------+--------------+---------+--------------------+---------------+--------------------+--------+-------------------+---------------+-------------+-------+---------+---------+--------+------------+--------------------+--------------------+
|Allocations|BrandingThemeID|             Contact|        CreditNoteID|CreditNoteNumber|CurrencyCode|CurrencyRate|                Date|         DateString|DueDate|DueDateString|FullyPaidOnDate|HasAttachments|HasErrors|                  ID|LineAmountTypes|           LineItems|Payments|          Reference|RemainingCredit|SentToContact| Status| SubTotal|    Total|TotalTax|        Type|      UpdatedDateUTC| ReplicationDateTime|
+-----------+---------------+--------------------+--------------------+----------------+------------+------------+--------------------+---------

In [34]:
df_exp_2 = df_exp.select(F.expr("jsons.*"))

In [35]:
df_exp_2.show()

+-----------+---------------+--------------------+--------------------+----------------+------------+------------+--------------------+-------------------+-------+-------------+---------------+--------------+---------+--------------------+---------------+--------------------+--------+-------------------+---------------+-------------+-------+---------+---------+--------+------------+--------------------+
|Allocations|BrandingThemeID|             Contact|        CreditNoteID|CreditNoteNumber|CurrencyCode|CurrencyRate|                Date|         DateString|DueDate|DueDateString|FullyPaidOnDate|HasAttachments|HasErrors|                  ID|LineAmountTypes|           LineItems|Payments|          Reference|RemainingCredit|SentToContact| Status| SubTotal|    Total|TotalTax|        Type|      UpdatedDateUTC|
+-----------+---------------+--------------------+--------------------+----------------+------------+------------+--------------------+-------------------+-------+-------------+---------